<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [2]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd

C:\Users\user\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Anaconda3\li

In [17]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [18]:
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

In [19]:
data = pd.read_csv("C:\\Users\\user\\Desktop\\Callbacks\\data.csv")
from sklearn.model_selection import train_test_split
X=data.loc[:,'f1':'f2']
Y=data.loc[:,'label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, stratify=Y)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(15000, 2)
(5000, 2)
(15000,)
(5000,)


In [20]:
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        print('train_begin')
        
    def on_epoch_end(self, epoch, logs={}):
        print('\n','epoch_end')
        # predict probabilities for test set
        yhat_probs = model.predict(X_test, verbose=0)
        # predict crisp classes for test set
        #yhat_classes = model.predict_proba(X_test, verbose=0)
        # reduce to 1d array
        yhat_probs = yhat_probs[:, 0]
        #yhat_classes = yhat_classes[:, 0]
        #f1 = f1_score(Y_test, yhat_classes)
        #print('F1 score: %f' % f1)
        #auc = roc_auc_score(Y_test, yhat_probs)
        #print('ROC AUC: %f' % auc)
        ## on end of each epoch, we will get logs and update the self.history dict
        
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        #print(self.history)
        #self.history['auc'].append(logs.get('auc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))
        #print('\n',self.history)

In [21]:

from tensorflow.keras.callbacks import EarlyStopping

In [22]:
class TerminateNaN(tf.keras.callbacks.Callback):
        
    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True

In [23]:
from tensorflow.keras.callbacks import LearningRateScheduler
def changeLearningRate(epoch,lr):
    if epoch==0:
        changed=0.01
        print('Initializing Epoch with value:',changed)
    if epoch>1: 
        val_acc_prev=history_own.history.get('val_acc')[epoch-2]
        val_acc_curr=history_own.history.get('val_acc')[epoch-1]
        if (val_acc_curr<val_acc_prev):
            changed=lr*0.9
            print('Val_acc is not increased,Decreasing epoch by 10%:',changed)

    if epoch%3==0 and epoch>1:
        changed = lr*0.95
        print(epoch,'th Epoch,Decreasing epoch by 5%:',changed)
    else:
        changed = lr  
    return changed

# Model-1

In [24]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=20))(input_layer)
layer2 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=30))(layer1)
layer3 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=40))(layer2)
layer4 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=50))(layer3)
layer5 = Dense(10,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=60))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=70))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
history_own = LossHistory()
optimizer = tf.keras.optimizers.SGD(0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint

#Callbacks
#file path, it saves the model in the 'model_save' folder and we are naming model with epoch number 
#and val acc to differtiate with other models
#you have to create model_save folder before running the code.
filepath="drive/My Drive/callbacks/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.35, patience=3, verbose=1)
optimizer = tf.keras.optimizers.Adam(0.01)
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
terminate=TerminateNaN()
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=20,validation_data=(X_test,Y_test),batch_size=10000,callbacks=[history_own,earlystop,terminate,lrschedule])

Train on 15000 samples, validate on 5000 samples
train_begin
Initializing Epoch with value: 0.01

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/20
10000/15000 [===================>..........] - ETA: 0s - loss: 0.7558 - acc: 0.4972
 epoch_end
15000/15000 [==============================] - 2s 153us/sample - loss: 0.7492 - acc: 0.4991 - val_loss: 0.7244 - val_acc: 0.5010

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/20
10000/15000 [===================>..........] - ETA: 0s - loss: 0.7275 - acc: 0.4946
 epoch_end
15000/15000 [==============================] - 0s 10us/sample - loss: 0.7218 - acc: 0.4991 - val_loss: 0.7048 - val_acc: 0.5018

Epoch 00003: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 3/20
10000/15000 [===================>..........] - ETA: 0s - loss: 0.7059 - acc: 0.4974
 epoch_end
15000/15000 [==============================] - 0s 9us/sample - loss: 0.7032 

# Model-2

In [26]:

#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=20))(input_layer)
layer2 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=30))(layer1)
layer3 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=40))(layer2)
layer4 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=50))(layer3)
layer5 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=60))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=70))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
history_own = LossHistory()
optimizer = tf.keras.optimizers.SGD(0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint

#Callbacks
#file path, it saves the model in the 'model_save' folder and we are naming model with epoch number 
#and val acc to differtiate with other models
#you have to create model_save folder before running the code.
filepath="drive/My Drive/callbacks/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.35, patience=3, verbose=1)
optimizer = tf.keras.optimizers.Adam(0.01)
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
terminate=TerminateNaN()
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=20,validation_data=(X_test,Y_test),batch_size=10000,callbacks=[history_own,earlystop,terminate,lrschedule])

Train on 15000 samples, validate on 5000 samples
train_begin
Initializing Epoch with value: 0.01

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/20
10000/15000 [===================>..........] - ETA: 0s - loss: 4.0433 - acc: 0.4987
 epoch_end
15000/15000 [==============================] - 3s 171us/sample - loss: 4.0856 - acc: 0.4983 - val_loss: 3.9458 - val_acc: 0.5000

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/20
10000/15000 [===================>..........] - ETA: 0s - loss: 3.9402 - acc: 0.4995
 epoch_end
15000/15000 [==============================] - 0s 11us/sample - loss: 2.9285 - acc: 0.5000 - val_loss: 3.7702 - val_acc: 0.5020

Epoch 00003: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 3/20
10000/15000 [===================>..........] - ETA: 0s - loss: 3.7419 - acc: 0.5054
 epoch_end
15000/15000 [==============================] - 0s 10us/sample - loss: 3.8218

# Model-3

In [30]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=20))(input_layer)
layer2 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=30))(layer1)
layer3 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=40))(layer2)
layer4 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=50))(layer3)
layer5 = Dense(10,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform(seed=60))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=70))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)
history_own = LossHistory()
optimizer = tf.keras.optimizers.SGD(0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
from tensorflow.keras.callbacks import ModelCheckpoint

#Callbacks
#file path, it saves the model inl the 'model_save' folder and we are naming model with epoch number 
#and val acc to differtiate with other models
#you have to create model_save folder before running the code.
filepath="drive/My Drive/callbacks/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.35, patience=3, verbose=1)
optimizer = tf.keras.optimizers.Adam(0.01)
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
terminate=TerminateNaN()
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs=20,validation_data=(X_test,Y_test),batch_size=10000,callbacks=[history_own,earlystop,terminate,lrschedule])

Train on 15000 samples, validate on 5000 samples
train_begin
Initializing Epoch with value: 0.01

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/20
10000/15000 [===================>..........] - ETA: 0s - loss: 0.8136 - acc: 0.5009
 epoch_end
15000/15000 [==============================] - 3s 192us/sample - loss: 0.7937 - acc: 0.4996 - val_loss: 0.7156 - val_acc: 0.4912

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/20
10000/15000 [===================>..........] - ETA: 0s - loss: 0.7200 - acc: 0.4689
 epoch_end
15000/15000 [==============================] - 0s 11us/sample - loss: 0.7136 - acc: 0.4661 - val_loss: 0.6943 - val_acc: 0.4636
Val_acc is not increased,Decreasing epoch by 10%: 0.008999999798834325

Epoch 00003: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 3/20
10000/15000 [===================>..........] - ETA: 0s - loss: 0.6952 - acc: 0.4625
 epoch_end
15000

In [32]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.
